<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/dsportfolio/Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Model Training](https://drive.google.com/uc?id=17cUxpbL2eonLzgY0uIVAxmsfMfQF1by2)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls '/content/drive/My Drive/Colab Notebooks/twitter.properties'

In [0]:
!pip install ConfigParser

[twitter]
accesstoken=xxxx

accesstokensecret=xxxx

apikey=xxxxxx

apisecretkey=xxxxxx

googleapikey=xxxxxxx

In [0]:
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitter.properties')

print(config.sections());


In [0]:
import os
import tweepy as tw

In [0]:
accesstoken=config.get('twitter', 'accesstoken')
accesstokensecret=config.get('twitter', 'accesstokensecret')
apikey=config.get('twitter', 'apikey')
apisecretkey=config.get('twitter', 'apisecretkey')

In [0]:
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
search_words = "#coronavirus"
date_since = "2020-03-10"

In [0]:
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1000)

In [0]:
tweets

In [0]:
tweet_details = [[tweet.geo, tweet.text,tweet.user.screen_name, tweet.user.location] for tweet in tweets]
#tweet_details

In [0]:
import pandas as pd

In [0]:
tweet_df = pd.DataFrame(data=tweet_details, columns=['geo','text','user', "location"])

In [0]:
pd.set_option('max_colwidth', 800)

tweet_df.head(20)

In [0]:
tweet_df.user.value_counts()

In [0]:
tweet_df.location.value_counts()

In [0]:
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)
    return text

In [0]:
tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

In [0]:
tweet_df.head(20)

In [0]:
tweet_df.to_csv('tweets.csv')

In [0]:
!ls

In [0]:
import spacy 
  
nlp = spacy.load('en_core_web_sm') 

In [0]:
tweet_df['text'].apply(lambda x: [print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) if (not ent.text.startswith('#')) else ""  for ent in nlp(x).ents])

In [0]:
tweet_df['entities']=tweet_df['text'].apply(lambda x: [(ent.text, ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(x).ents])

In [0]:
tweet_df.head(20)

In [0]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [0]:
sid = SentimentIntensityAnalyzer()

In [0]:
tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [0]:
tweet_df.head(10)

In [0]:
!pip install googlemaps

In [0]:
import googlemaps

In [0]:
gmaps = googlemaps.Client(key=config.get('twitter', 'googleapikey'))

geocode_result = gmaps.geocode(tweet_df['location'][1])

print(geocode_result)
print(geocode_result[0]['formatted_address'])

print(geocode_result[0]['formatted_address'].split(",")[-1].strip())



In [0]:
def get_country(input):
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output

In [0]:
tweet_df['country']=tweet_df['location'].apply(lambda x: "" if (not x.strip()) else get_country(x))

In [0]:
tweet_df['country'].value_counts()

In [0]:
tweet_df.head(10)

In [0]:
tweet_df.dtypes

In [0]:
tweet_df.to_csv("tweets.csv")